# Regra de negócios aplicada ao Olist Dataset

A regra de negócios neste notebook é uma simulação para extrair insights dos dados da camada Silver e adiciona-los à camada Gold deste projeto. Este notebook permitirá construir dashboards e relatórios posteriormente.

## Business Rules Checklist

Abaixo temos um checklist de regras de negócio simulado, sugestões de o que será extraído ao decorrer deste notebook, sabendo disso, segue a lista do checklist da regra de negócios:

Com base na estrutura dos datasets:


📌 2. **Vendedores (sellers)**

    ✅ Regra de Negócio: Vendedores com alta taxa de cancelamento ou avaliações negativas devem passar por auditoria e treinamentos.

📌 3. **Clientes (customers)**

    ✅ Regra de Negócio: Oferecer promoções direcionadas para clientes de regiões com alto ticket médio para aumentar recorrência de compras.

📌 4. **Geolocalização (geolocation)**

    ✅ Regra de Negócio: Expandir vendedores para regiões com alta demanda e poucos fornecedores para equilibrar a oferta de produtos.

📌 5. **Itens dos Pedidos (order_items)**

    ✅ Regra de Negócio: Produtos com baixa margem de lucro e alto custo de frete devem ser reavaliados no catálogo.

📌 6. **Pagamentos (order_payments)**

    ✅ Regra de Negócio: Estimular métodos de pagamento que geram maior ticket médio e menor taxa de cancelamento.

📌 7. **Avaliações (order_reviews)**

    ✅ Regra de Negócio: Vendedores com avaliações abaixo de 3 estrelas devem ser notificados para ações corretivas.

📌 8. **Pedidos (orders)**

    ✅ Regra de Negócio: Monitorar pedidos com atraso recorrente e penalizar transportadoras que descumprirem prazos.

📌 9. **Produtos (products)**

    ✅ Regra de Negócio: Produtos com maior volume de vendas devem ter prioridade no estoque para evitar rupturas.

📌 10. **Leads Qualificados (leads_qualified)**

    ✅ Regra de Negócio: Leads qualificados com origem em campanhas de alto desempenho devem receber prioridade no atendimento.

In [1]:
import os
import logging
import duckdb
from deltalake import write_deltalake

# Configuração do logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

def gold_pipeline_sql(query: str, gold_table_name: str, mode: str = "overwrite"):
    """Executa uma query SQL sobre um Delta Table Silver e salva o resultado na camada Gold."""
    
    gold_path = "../delta_lake/gold/"
    gold_path_delta = f"{gold_path}{gold_table_name}"
    os.makedirs(gold_path, exist_ok=True) # cria o diretório se não existir

    try:
        with duckdb.connect() as conn:
            logger.info(f"Executando transformação na tabela '{gold_table_name}' para a camada GOLD")
            df_transformed_gold = conn.sql(query).arrow()
            write_deltalake(gold_path_delta, df_transformed_gold, mode=mode)
            logger.info(f"\033[32m[OK]\033[0m Tabela '{gold_table_name}' processada com sucesso.")

    except Exception as e:
        logger.error(f"\033[31m[ERROR]\033[0m Erro inesperado ao processar '{gold_table_name}': {str(e)}")

    finally:
        if conn:
            conn.close()

In [2]:
import pandas as pd
import duckdb

def pandas_sql(query: str) -> pd.DataFrame:
    """Executa uma consulta SQL e retorna um DataFrame."""
    conn = duckdb.connect()
    result = conn.sql(query).fetchdf()
    conn.close()
    return result

## Vendedores Rule

✅ Business Rule: Salespeople with a high cancellation rate or negative reviews must undergo auditing and training.

✅ Regra de Negócio: Vendedores com alta taxa de cancelamento ou avaliações negativas devem passar por auditoria e treinamentos.

**Category to each metric**

1. Cancellations (Cancelamentos)
- **>= 20%** → HIGH  
- **10% a 20%** → MEDIAN  
- **< 10%** → LOW

2. Negative_reviews (Avaliações negativas)
- **>= 30%** → HIGH  
- **15% a 30%** → MEDIAN  
- **< 15%** → LOW  

3. Delivery Delays - in days (Atrasos de entrega - em dias)
- **>= 5 dias** → HIGH  
- **2 to 5 dias** → MEDIAN  
- **< 2 dias** → LOW  

4. Seller lifetime (tempo de atuação dos vendedores)
- **< 30 dias** → BEGINNER (Inciante)
- **30 to 180 dias** → NEWBIE (Novato)  
- **> 180 dias** → EXPERT (Experiente)

5. Total risk score (Classificação final)
- **Score 10 to 12** → HIGH  
- **Score 7 to 9** → MEDIAN  
- **Score up to 6** → LOW

In [22]:
# pipeline de dados na camada gold que realiza a consulta SQL e gera uma nova tabela para visualização posterior
gold_pipeline_sql(query="""
WITH pedidos_por_vendedor AS (
    SELECT 
        oi.seller_id,
        o.order_id,
        o.order_status
    FROM delta_scan('../delta_lake/silver/order_items_silver') oi
    JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o ON o.order_id = oi.order_id
),
reviews_por_vendedor AS (
    SELECT 
        oi.seller_id,
        r.review_score
    FROM delta_scan('../delta_lake/silver/order_items_silver') oi
    JOIN delta_scan('../delta_lake/silver/order_reviews') r ON r.order_id = oi.order_id
),
agg_cancellation AS (
    SELECT 
        seller_id,
        COUNT(*) AS total_orders,
        SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) AS canceled_orders,
        ROUND(SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS cancellation_rate
    FROM pedidos_por_vendedor
    GROUP BY seller_id
),
agg_reviews AS (
    SELECT 
        seller_id,
        COUNT(*) AS total_reviews,
        SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) AS negative_reviews,
        ROUND(SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS negative_review_rate
    FROM reviews_por_vendedor
    GROUP BY seller_id
)

SELECT 
    c.seller_id,
    c.total_orders,
    c.canceled_orders,
    c.cancellation_rate,
    r.total_reviews,
    r.negative_reviews,
    r.negative_review_rate
FROM agg_cancellation c
LEFT JOIN agg_reviews r ON c.seller_id = r.seller_id
ORDER BY cancellation_rate DESC, negative_review_rate DESC;
                  
""", gold_table_name="agg_sellers_cancellation_rate")

2025-04-06 11:38:33,266 - INFO - Executando transformação na tabela 'agg_sellers_cancellation_rate' para a camada GOLD
2025-04-06 11:38:34,943 - INFO - [OK] Tabela 'agg_sellers_cancellation_rate' processada com sucesso.


In [ ]:
# Análise SQL sobre os dados silver para visualizar os dados gold
pandas_sql("""
WITH pedidos_por_vendedor AS (
    SELECT 
        oi.seller_id,
        o.order_id,
        o.order_status
    FROM delta_scan('../delta_lake/silver/order_items_silver') oi
    JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o ON o.order_id = oi.order_id
),
reviews_por_vendedor AS (
    SELECT 
        oi.seller_id,
        r.review_score
    FROM delta_scan('../delta_lake/silver/order_items_silver') oi
    JOIN delta_scan('../delta_lake/silver/order_reviews') r ON r.order_id = oi.order_id
),
agg_cancellation AS (
    SELECT 
        seller_id,
        COUNT(*) AS total_orders,
        SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) AS canceled_orders,
        ROUND(SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS cancellation_rate
    FROM pedidos_por_vendedor
    GROUP BY seller_id
),
agg_reviews AS (
    SELECT 
        seller_id,
        COUNT(*) AS total_reviews,
        SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) AS negative_reviews,
        ROUND(SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS negative_review_rate
    FROM reviews_por_vendedor
    GROUP BY seller_id
)

SELECT 
    c.seller_id,
    c.total_orders,
    c.canceled_orders,
    c.cancellation_rate,
    r.total_reviews,
    r.negative_reviews,
    r.negative_review_rate
FROM agg_cancellation c
LEFT JOIN agg_reviews r ON c.seller_id = r.seller_id
ORDER BY cancellation_rate DESC, negative_review_rate DESC LIMIT 10;
""")

,seller_id,total_orders,canceled_orders,cancellation_rate,total_reviews,negative_reviews,negative_review_rate
0,92992bec39dde5766f590d3b1f46d465,1,1.0,1.0,1,1.0,1.0
1,bf0d50a6410d487dc97d2baac0a8c0be,1,1.0,1.0,1,1.0,1.0
2,66d5ae34537110d3affc56174dad2b2f,3,3.0,1.0,3,3.0,1.0
3,f09e26011d833ddab11593c1a097a92a,1,1.0,1.0,1,1.0,1.0
4,afac9f6b0c4e5f0a7ee87e922cd0f873,1,1.0,1.0,1,1.0,1.0
5,6e85dc5ecd97a61094b89b046a509d8e,1,1.0,1.0,1,1.0,1.0
6,278b6e0b20c4f61fefaa0577943d7a35,2,2.0,1.0,2,2.0,1.0
7,1fddcb7b326905d3ad5efe82187db347,1,1.0,1.0,1,1.0,1.0
8,58b98ccb79873e04eac4357cacc590d9,1,1.0,1.0,1,1.0,1.0
9,fa18c782a4866d313cadab567097f8f1,1,1.0,1.0,1,1.0,1.0


In [28]:
# filtrando somente os vendedores com risco elevado, de acordo com os seguintes critérios
# cancelamento acima de 10% ou avaliações negativas acima de 20%
pandas_sql("""
    SELECT 
        sc.seller_id,
        sc.total_orders,
        sc.canceled_orders,
        sc.cancellation_rate,
        sc.total_reviews,
        sc.negative_reviews,
        sc.negative_review_rate
    FROM delta_scan('../delta_lake/gold/agg_sellers_cancellation_rate') sc
           
    -- filtering only sellers with high risk, according to the following criteria
    -- cancellations over 10% or negative reviews over 20%
    WHERE sc.cancellation_rate > 0.1 OR sc.negative_review_rate > 0.2 LIMIT 5;
""")

,seller_id,total_orders,canceled_orders,cancellation_rate,total_reviews,negative_reviews,negative_review_rate
0,a4e634166243bbb3a13fa584463fa468,2,2.0,1.0,2,2.0,1.0
1,66d5ae34537110d3affc56174dad2b2f,3,3.0,1.0,3,3.0,1.0
2,58b98ccb79873e04eac4357cacc590d9,1,1.0,1.0,1,1.0,1.0
3,1fddcb7b326905d3ad5efe82187db347,1,1.0,1.0,1,1.0,1.0
4,278b6e0b20c4f61fefaa0577943d7a35,2,2.0,1.0,2,2.0,1.0


In [32]:
# realizando a mesma consulta, porém com uma MÉTRICA COMPOSTA
# “pesando” mais o cancelamento se ele for mais grave no negócio.
pandas_sql("""
    SELECT 
        sc.seller_id,
        sc.total_orders,
        sc.canceled_orders,
        sc.cancellation_rate,
        sc.total_reviews,
        sc.negative_reviews,
        sc.negative_review_rate
    FROM delta_scan('../delta_lake/gold/agg_sellers_cancellation_rate') sc
           
    -- performing the same query, but with a COMPOSITE METRIC
    -- “weighting” the cancellation more heavily if it is more serious in business.
    ORDER BY (sc.cancellation_rate * 2 + sc.negative_review_rate) DESC LIMIT 5;
""")

,seller_id,total_orders,canceled_orders,cancellation_rate,total_reviews,negative_reviews,negative_review_rate
0,1fddcb7b326905d3ad5efe82187db347,1,1.0,1.0,1,1.0,1.0
1,66d5ae34537110d3affc56174dad2b2f,3,3.0,1.0,3,3.0,1.0
2,278b6e0b20c4f61fefaa0577943d7a35,2,2.0,1.0,2,2.0,1.0
3,58b98ccb79873e04eac4357cacc590d9,1,1.0,1.0,1,1.0,1.0
4,a4e634166243bbb3a13fa584463fa468,2,2.0,1.0,2,2.0,1.0


In [40]:
# Análise SQL sobre os dados silver para visualizar os dados gold - V2
# acrescentando mais algumas tabelas:
# avg_delivery_delay
# agg_seller_lifetime
# score
# risk_level
pandas_sql("""
WITH pedidos_por_vendedor AS (
    SELECT 
        oi.seller_id,
        o.order_id,
        o.order_status,
        o.order_purchase_timestamp,
        o.order_delivered_customer_date,
        o.order_estimated_delivery_date
    FROM delta_scan('../delta_lake/silver/order_items_silver') oi
    JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o 
        ON o.order_id = oi.order_id
),

reviews_por_vendedor AS (
    SELECT 
        oi.seller_id,
        r.review_score
    FROM delta_scan('../delta_lake/silver/order_items_silver') oi
    JOIN delta_scan('../delta_lake/silver/order_reviews') r 
        ON r.order_id = oi.order_id
),

agg_cancellation AS (
    SELECT 
        seller_id,
        COUNT(*) AS total_orders,
        SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) AS canceled_orders,
        ROUND(SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS cancellation_rate
    FROM pedidos_por_vendedor
    GROUP BY seller_id
),

agg_reviews AS (
    SELECT 
        seller_id,
        COUNT(*) AS total_reviews,
        SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) AS negative_reviews,
        ROUND(SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS negative_review_rate
    FROM reviews_por_vendedor
    GROUP BY seller_id
),

agg_delivery_delay AS (
    SELECT 
        seller_id,
        ROUND(AVG(CAST(julian(order_delivered_customer_date) - julian(order_estimated_delivery_date) AS FLOAT)), 2) AS avg_delivery_delay
    FROM pedidos_por_vendedor
    WHERE order_delivered_customer_date IS NOT NULL
      AND order_estimated_delivery_date IS NOT NULL
    GROUP BY seller_id
),

agg_seller_lifetime AS (
    SELECT 
        seller_id,
        CAST(julian(MAX(order_purchase_timestamp)) - julian(MIN(order_purchase_timestamp)) AS INT) AS seller_lifetime_days
    FROM pedidos_por_vendedor
    GROUP BY seller_id
),

final_score AS (
    SELECT 
        c.seller_id,
        c.total_orders,
        c.canceled_orders,
        c.cancellation_rate,
        r.total_reviews,
        r.negative_reviews,
        r.negative_review_rate,
        d.avg_delivery_delay,
        l.seller_lifetime_days,

        -- Scoring por métrica
        CASE 
            WHEN c.cancellation_rate >= 0.2 THEN 3
            WHEN c.cancellation_rate >= 0.1 THEN 2
            ELSE 1
        END AS score_cancellation,

        CASE 
            WHEN r.negative_review_rate >= 0.3 THEN 3
            WHEN r.negative_review_rate >= 0.15 THEN 2
            ELSE 1
        END AS score_reviews,

        CASE 
            WHEN d.avg_delivery_delay >= 5 THEN 3
            WHEN d.avg_delivery_delay >= 2 THEN 2
            ELSE 1
        END AS score_delay,

        CASE 
            WHEN l.seller_lifetime_days < 30 THEN 1
            ELSE 0
        END AS score_inexperience,

        -- Soma dos scores
        (
            CASE 
                WHEN c.cancellation_rate >= 0.2 THEN 3
                WHEN c.cancellation_rate >= 0.1 THEN 2
                ELSE 1
            END
            +
            CASE 
                WHEN r.negative_review_rate >= 0.3 THEN 3
                WHEN r.negative_review_rate >= 0.15 THEN 2
                ELSE 1
            END
            +
            CASE 
                WHEN d.avg_delivery_delay >= 5 THEN 3
                WHEN d.avg_delivery_delay >= 2 THEN 2
                ELSE 1
            END
            +
            CASE 
                WHEN l.seller_lifetime_days < 30 THEN 1
                ELSE 0
            END
        ) AS total_risk_score,

        -- Classificação de risco
        CASE 
            WHEN (
                CASE 
                    WHEN c.cancellation_rate >= 0.2 THEN 3
                    WHEN c.cancellation_rate >= 0.1 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN r.negative_review_rate >= 0.3 THEN 3
                    WHEN r.negative_review_rate >= 0.15 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN d.avg_delivery_delay >= 5 THEN 3
                    WHEN d.avg_delivery_delay >= 2 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN l.seller_lifetime_days < 30 THEN 1
                    ELSE 0
                END
            ) >= 10 THEN 'HIGH'
            WHEN (
                CASE 
                    WHEN c.cancellation_rate >= 0.2 THEN 3
                    WHEN c.cancellation_rate >= 0.1 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN r.negative_review_rate >= 0.3 THEN 3
                    WHEN r.negative_review_rate >= 0.15 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN d.avg_delivery_delay >= 5 THEN 3
                    WHEN d.avg_delivery_delay >= 2 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN l.seller_lifetime_days < 30 THEN 1
                    ELSE 0
                END
            ) >= 7 THEN 'MEDIAN'
            ELSE 'LOW'
        END AS risk_level

    FROM agg_cancellation c
    LEFT JOIN agg_reviews r ON c.seller_id = r.seller_id
    LEFT JOIN agg_delivery_delay d ON c.seller_id = d.seller_id
    LEFT JOIN agg_seller_lifetime l ON c.seller_id = l.seller_id
)

SELECT * 
FROM final_score
ORDER BY total_risk_score DESC, cancellation_rate DESC
LIMIT 10;
""")

,seller_id,total_orders,canceled_orders,cancellation_rate,total_reviews,negative_reviews,negative_review_rate,avg_delivery_delay,seller_lifetime_days,score_cancellation,score_reviews,score_delay,score_inexperience,total_risk_score,risk_level
0,c004e5ea15737026cecaee0447e00b75,2,1.0,0.5000,2,2.0,1.0000,15.84,4,3,3,3,1,10,HIGH
1,c1d1ebb4745cc43cb6f05e9c8a705c9a,3,2.0,0.6667,3,2.0,0.6667,5.40,211,3,3,3,0,9,MEDIAN
2,e5bd51692380fd8cabb19b756b95e714,3,1.0,0.3333,3,2.0,0.6667,6.16,136,3,3,3,0,9,MEDIAN
3,312ba1d77e9c332ef21f9598b7f64cd7,10,3.0,0.3000,11,6.0,0.5455,5.44,175,3,3,3,0,9,MEDIAN
4,eebb3372362aa9a46975164bed19a7e7,8,2.0,0.2500,7,4.0,0.5714,27.66,56,3,3,3,0,9,MEDIAN
5,6e85dc5ecd97a61094b89b046a509d8e,1,1.0,1.0000,1,1.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN
6,afac9f6b0c4e5f0a7ee87e922cd0f873,1,1.0,1.0000,1,1.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN
7,92992bec39dde5766f590d3b1f46d465,1,1.0,1.0000,1,1.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN
8,8501d82f68d23148b6d78bb7c4a42037,1,1.0,1.0000,1,1.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN
9,37ceb404da208418c9032f0674e57804,3,3.0,1.0000,3,3.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN


In [41]:
gold_pipeline_sql(query="""
WITH pedidos_por_vendedor AS (
    SELECT 
        oi.seller_id,
        o.order_id,
        o.order_status,
        o.order_purchase_timestamp,
        o.order_delivered_customer_date,
        o.order_estimated_delivery_date
    FROM delta_scan('../delta_lake/silver/order_items_silver') oi
    JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o 
        ON o.order_id = oi.order_id
),

reviews_por_vendedor AS (
    SELECT 
        oi.seller_id,
        r.review_score
    FROM delta_scan('../delta_lake/silver/order_items_silver') oi
    JOIN delta_scan('../delta_lake/silver/order_reviews') r 
        ON r.order_id = oi.order_id
),

agg_cancellation AS (
    SELECT 
        seller_id,
        COUNT(*) AS total_orders,
        SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) AS canceled_orders,
        ROUND(SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS cancellation_rate
    FROM pedidos_por_vendedor
    GROUP BY seller_id
),

agg_reviews AS (
    SELECT 
        seller_id,
        COUNT(*) AS total_reviews,
        SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) AS negative_reviews,
        ROUND(SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS negative_review_rate
    FROM reviews_por_vendedor
    GROUP BY seller_id
),

agg_delivery_delay AS (
    SELECT 
        seller_id,
        ROUND(AVG(CAST(julian(order_delivered_customer_date) - julian(order_estimated_delivery_date) AS FLOAT)), 2) AS avg_delivery_delay
    FROM pedidos_por_vendedor
    WHERE order_delivered_customer_date IS NOT NULL
      AND order_estimated_delivery_date IS NOT NULL
    GROUP BY seller_id
),

agg_seller_lifetime AS (
    SELECT 
        seller_id,
        CAST(julian(MAX(order_purchase_timestamp)) - julian(MIN(order_purchase_timestamp)) AS INT) AS seller_lifetime_days
    FROM pedidos_por_vendedor
    GROUP BY seller_id
),

final_score AS (
    SELECT 
        c.seller_id,
        c.total_orders,
        c.canceled_orders,
        c.cancellation_rate,
        r.total_reviews,
        r.negative_reviews,
        r.negative_review_rate,
        d.avg_delivery_delay,
        l.seller_lifetime_days,

        -- Scoring por métrica
        CASE 
            WHEN c.cancellation_rate >= 0.2 THEN 3
            WHEN c.cancellation_rate >= 0.1 THEN 2
            ELSE 1
        END AS score_cancellation,

        CASE 
            WHEN r.negative_review_rate >= 0.3 THEN 3
            WHEN r.negative_review_rate >= 0.15 THEN 2
            ELSE 1
        END AS score_reviews,

        CASE 
            WHEN d.avg_delivery_delay >= 5 THEN 3
            WHEN d.avg_delivery_delay >= 2 THEN 2
            ELSE 1
        END AS score_delay,

        CASE 
            WHEN l.seller_lifetime_days < 30 THEN 1
            ELSE 0
        END AS score_inexperience,

        -- Soma dos scores
        (
            CASE 
                WHEN c.cancellation_rate >= 0.2 THEN 3
                WHEN c.cancellation_rate >= 0.1 THEN 2
                ELSE 1
            END
            +
            CASE 
                WHEN r.negative_review_rate >= 0.3 THEN 3
                WHEN r.negative_review_rate >= 0.15 THEN 2
                ELSE 1
            END
            +
            CASE 
                WHEN d.avg_delivery_delay >= 5 THEN 3
                WHEN d.avg_delivery_delay >= 2 THEN 2
                ELSE 1
            END
            +
            CASE 
                WHEN l.seller_lifetime_days < 30 THEN 1
                ELSE 0
            END
        ) AS total_risk_score,

        -- Classificação de risco
        CASE 
            WHEN (
                CASE 
                    WHEN c.cancellation_rate >= 0.2 THEN 3
                    WHEN c.cancellation_rate >= 0.1 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN r.negative_review_rate >= 0.3 THEN 3
                    WHEN r.negative_review_rate >= 0.15 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN d.avg_delivery_delay >= 5 THEN 3
                    WHEN d.avg_delivery_delay >= 2 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN l.seller_lifetime_days < 30 THEN 1
                    ELSE 0
                END
            ) >= 10 THEN 'HIGH'
            WHEN (
                CASE 
                    WHEN c.cancellation_rate >= 0.2 THEN 3
                    WHEN c.cancellation_rate >= 0.1 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN r.negative_review_rate >= 0.3 THEN 3
                    WHEN r.negative_review_rate >= 0.15 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN d.avg_delivery_delay >= 5 THEN 3
                    WHEN d.avg_delivery_delay >= 2 THEN 2
                    ELSE 1
                END
                +
                CASE 
                    WHEN l.seller_lifetime_days < 30 THEN 1
                    ELSE 0
                END
            ) >= 7 THEN 'MEDIAN'
            ELSE 'LOW'
        END AS risk_level

    FROM agg_cancellation c
    LEFT JOIN agg_reviews r ON c.seller_id = r.seller_id
    LEFT JOIN agg_delivery_delay d ON c.seller_id = d.seller_id
    LEFT JOIN agg_seller_lifetime l ON c.seller_id = l.seller_id
)

SELECT * 
FROM final_score
ORDER BY total_risk_score DESC, cancellation_rate DESC
""", gold_table_name="agg_sellers_cancellation_rate_v2")

2025-04-06 12:19:05,575 - INFO - Executando transformação na tabela 'agg_sellers_cancellation_rate_v2' para a camada GOLD
2025-04-06 12:19:05,771 - INFO - [OK] Tabela 'agg_sellers_cancellation_rate_v2' processada com sucesso.


In [43]:
pandas_sql("SELECT * FROM delta_scan('../delta_lake/gold/agg_sellers_cancellation_rate_v2') LIMIT 10")

,seller_id,total_orders,canceled_orders,cancellation_rate,total_reviews,negative_reviews,negative_review_rate,avg_delivery_delay,seller_lifetime_days,score_cancellation,score_reviews,score_delay,score_inexperience,total_risk_score,risk_level
0,c004e5ea15737026cecaee0447e00b75,2,1.0,0.5000,2,2.0,1.0000,15.84,4,3,3,3,1,10,HIGH
1,c1d1ebb4745cc43cb6f05e9c8a705c9a,3,2.0,0.6667,3,2.0,0.6667,5.40,211,3,3,3,0,9,MEDIAN
2,e5bd51692380fd8cabb19b756b95e714,3,1.0,0.3333,3,2.0,0.6667,6.16,136,3,3,3,0,9,MEDIAN
3,312ba1d77e9c332ef21f9598b7f64cd7,10,3.0,0.3000,11,6.0,0.5455,5.44,175,3,3,3,0,9,MEDIAN
4,eebb3372362aa9a46975164bed19a7e7,8,2.0,0.2500,7,4.0,0.5714,27.66,56,3,3,3,0,9,MEDIAN
5,37ceb404da208418c9032f0674e57804,3,3.0,1.0000,3,3.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN
6,8501d82f68d23148b6d78bb7c4a42037,1,1.0,1.0000,1,1.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN
7,bf0d50a6410d487dc97d2baac0a8c0be,1,1.0,1.0000,1,1.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN
8,5dc4bdaf869e8c77c648078cbe8d59c2,1,1.0,1.0000,1,1.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN
9,dadc51ef321949ec9a3ab25cd902e23d,1,1.0,1.0000,1,1.0,1.0000,NaN,0,3,3,1,1,8,MEDIAN


## Clientes Rule

✅ Business Rule: Offer targeted promotions to customers in regions with a high average ticket to increase recurrent purchases.

✅ Regra de Negócio: Oferecer promoções direcionadas para clientes de regiões com alto ticket médio para aumentar recorrência de compras.

In [44]:
pandas_sql("SELECT * FROM delta_scan('../delta_lake/silver/aggregated_customers') LIMIT 5")

,customer_id,customer_unique_id,customer_cep,customer_city,customer_state,avg_latitude,avg_longitude
0,39b95a2690bce9433140be3c4cdcce38,f29c287e3867744b35079cc697f01935,02462,sao paulo,SP,-23.491800,-46.641607
1,f8125bb662b6ef384d35dd00b3d64c9d,0e02d417b7a168e2916625d2cc43482f,02420,sao paulo,SP,-23.474187,-46.636394
2,2160421cae73963d7d4842ca9f340aff,ee5bb9c5ec9d199a7fa233463df461f0,02443,sao paulo,SP,-23.472995,-46.647429
3,9a29aa642b3884b48ea5a04729ae703c,b4b02dbd03241e9def68e7be74202bd6,02415,sao paulo,SP,-23.471948,-46.631370
4,624089fe851f3deb7223ffca03a829c8,56bf02d284b41876f8ceedda0ea1e1e9,02730,sao paulo,SP,-23.497797,-46.690799


In [46]:
pandas_sql("SELECT * FROM delta_scan('../delta_lake/silver/orders_full_data_silver') LIMIT 5")

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [47]:
pandas_sql("SELECT * FROM delta_scan('../delta_lake/silver/order_items_silver') LIMIT 5")

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [53]:
# criando consulta SQL para visualizar o gold_customers_segmented table
pandas_sql("""
WITH customer_orders AS (
    SELECT
        o.customer_id,
        o.order_id,
        o.order_purchase_timestamp
    FROM delta_scan('../delta_lake/silver/orders_full_data_silver') o
),

payments_per_order AS (
    SELECT
        op.order_id,
        SUM(op.payment_value) AS total_payment
    FROM delta_scan('../delta_lake/silver/payments_silver') op
    GROUP BY op.order_id
),

orders_with_payment AS (
    SELECT
        co.customer_id,
        co.order_id,
        co.order_purchase_timestamp,
        ppo.total_payment
    FROM customer_orders co
    LEFT JOIN payments_per_order ppo ON co.order_id = ppo.order_id
),

agg_metrics_per_customer AS (
    SELECT
        customer_id,
        COUNT(order_id) AS total_orders,
        SUM(total_payment) AS total_spent,
        AVG(total_payment) AS avg_ticket,
        MAX(order_purchase_timestamp) AS last_order_date,
        MIN(order_purchase_timestamp) AS first_order_date
    FROM orders_with_payment
    GROUP BY customer_id
)

SELECT
    ac.customer_id,
    ac.customer_cep AS customer_cep,
    ac.avg_latitude,
    ac.avg_longitude,
    amc.total_orders,
    amc.total_spent,
    amc.avg_ticket,
    amc.first_order_date,
    amc.last_order_date
FROM delta_scan('../delta_lake/silver/aggregated_customers') ac
LEFT JOIN agg_metrics_per_customer amc
    ON ac.customer_id = amc.customer_id
LIMIT 10;

""")

,customer_id,customer_cep,avg_latitude,avg_longitude,total_orders,total_spent,avg_ticket,first_order_date,last_order_date
0,0157d2d9039fc91ade6a389d733d8a16,02403,-23.493992,-46.628359,1,56.77,56.77,2018-05-09 13:44:18,2018-05-09 13:44:18
1,24c4a139416d9ba7fc593a3dc162df8b,02404,-23.491261,-46.628982,1,35.77,35.77,2018-03-11 11:38:50,2018-03-11 11:38:50
2,1de595522022ac1c1c7561673a6d71ef,02461,-23.492003,-46.639003,1,79.41,79.41,2018-03-07 17:35:31,2018-03-07 17:35:31
3,1a634ee44caf584f4374b84806146bcf,02404,-23.491261,-46.628982,1,154.74,154.74,2017-10-22 22:12:47,2017-10-22 22:12:47
4,83be2d70d23dff8950d003f73cc7fcaa,02422,-23.471749,-46.636773,1,284.90,284.90,2017-12-15 11:35:49,2017-12-15 11:35:49
5,e6746025c842eea668be9825f0476359,02403,-23.493992,-46.628359,1,94.66,94.66,2018-08-24 18:47:44,2018-08-24 18:47:44
6,25033515ca54549ab33a66e549c6f2de,02422,-23.471749,-46.636773,1,28.38,28.38,2017-09-29 00:54:27,2017-09-29 00:54:27
7,f980a49c5ca07a32a7958a75b6d190d0,02422,-23.471749,-46.636773,1,133.84,133.84,2018-07-17 02:07:26,2018-07-17 02:07:26
8,a881d768651dbf7f348ee3eb81e18139,02435,-23.480303,-46.641768,1,21.38,21.38,2018-06-09 19:07:39,2018-06-09 19:07:39
9,24bb86ce8ea9406d2e034154063030d4,02723,-23.499879,-46.680030,1,113.35,113.35,2018-04-26 14:10:07,2018-04-26 14:10:07


**Conclusão do exemplo**

cliente id = 0157d2d9039fc91ade6a389d733d8a16:

1. Fez apenas 1 pedido → baixa recorrência.

2. Gastou relativamente pouco (ticket médio = R$ 56,77).

3. Está em uma região com CEP 02403, que podemos mais tarde comparar com outros CEPs.

🔍 **Ou seja**: esse cliente não seria um bom alvo inicial para campanhas de fidelização baseadas em ticket alto e frequência. Mas pode ser útil em campanhas para recuperar clientes inativos.

In [54]:
gold_pipeline_sql(query="""
WITH customer_orders AS (
    SELECT
        o.customer_id,
        o.order_id,
        o.order_purchase_timestamp
    FROM delta_scan('../delta_lake/silver/orders_full_data_silver') o
),

payments_per_order AS (
    SELECT
        op.order_id,
        SUM(op.payment_value) AS total_payment
    FROM delta_scan('../delta_lake/silver/payments_silver') op
    GROUP BY op.order_id
),

orders_with_payment AS (
    SELECT
        co.customer_id,
        co.order_id,
        co.order_purchase_timestamp,
        ppo.total_payment
    FROM customer_orders co
    LEFT JOIN payments_per_order ppo ON co.order_id = ppo.order_id
),

agg_metrics_per_customer AS (
    SELECT
        customer_id,
        COUNT(order_id) AS total_orders,
        SUM(total_payment) AS total_spent,
        AVG(total_payment) AS avg_ticket,
        MAX(order_purchase_timestamp) AS last_order_date,
        MIN(order_purchase_timestamp) AS first_order_date
    FROM orders_with_payment
    GROUP BY customer_id
)

SELECT
    ac.customer_id,
    ac.customer_cep AS customer_cep,
    ac.avg_latitude,
    ac.avg_longitude,
    amc.total_orders,
    amc.total_spent,
    amc.avg_ticket,
    amc.first_order_date,
    amc.last_order_date
FROM delta_scan('../delta_lake/silver/aggregated_customers') ac
LEFT JOIN agg_metrics_per_customer amc
    ON ac.customer_id = amc.customer_id;
""", gold_table_name="gold_customers_segmented")

2025-04-06 21:48:43,395 - INFO - Executando transformação na tabela 'gold_customers_segmented' para a camada GOLD
2025-04-06 21:48:43,631 - INFO - [OK] Tabela 'gold_customers_segmented' processada com sucesso.


In [ ]:
# Rodar query para mais registros e agrupar por CEP (zip_code_prefix) para ver quais regiões têm:
# Maior avg_ticket.
# Maior total_orders médio por cliente.
# Mais clientes ativos.

# Identificar regiões com maior potencial de faturamento.
# Saber onde vale a pena focar campanhas de incentivo à recorrência.

# Gerar indicadores por região
pandas_sql("""
SELECT
    customer_cep,
    COUNT(DISTINCT customer_id) AS num_customers,
    AVG(avg_ticket) AS region_avg_ticket,
    AVG(total_orders) AS region_avg_orders
FROM delta_scan('../delta_lake/gold/gold_customers_segmented')
GROUP BY customer_cep
ORDER BY region_avg_ticket DESC
LIMIT 10;
""")

,customer_cep,num_customers,region_avg_ticket,region_avg_orders
0,38447,1,2416.00,1.0
1,02635,1,2328.40,1.0
2,58765,1,2324.99,1.0
3,57275,1,2269.98,1.0
4,74455,1,2267.28,1.0
5,68618,1,2252.66,1.0
6,79906,1,2133.69,1.0
7,39363,1,2106.55,1.0
8,77818,1,2094.44,1.0
9,55495,1,2066.34,1.0


In [57]:
# Indicadores por região (agrupado por customer_cep)
pandas_sql("""
SELECT
    customer_cep,
    COUNT(DISTINCT customer_id) AS num_customers,
    AVG(avg_ticket) AS region_avg_ticket,
    AVG(total_orders) AS region_avg_orders,
    SUM(total_spent) AS region_total_spent
FROM delta_scan('../delta_lake/gold/gold_customers_segmented')
GROUP BY customer_cep
ORDER BY region_avg_ticket DESC
LIMIT 10;
""")

# Essa visão já ajuda a responder perguntas como:
# Quais regiões têm ticket médio mais alto?
# Onde os clientes fazem mais pedidos, em média?
# Qual região tem maior volume total de compras?

# Com base nisso, podemos:
# Priorizar essas regiões para campanhas.
# Combinar com dados de categoria de produto depois (opcional).
# Estimar retorno de campanhas regionais.


,customer_cep,num_customers,region_avg_ticket,region_avg_orders,region_total_spent
0,38447,1,2416.00,1.0,2416.00
1,02635,1,2328.40,1.0,2328.40
2,58765,1,2324.99,1.0,2324.99
3,57275,1,2269.98,1.0,2269.98
4,74455,1,2267.28,1.0,2267.28
5,68618,1,2252.66,1.0,2252.66
6,79906,1,2133.69,1.0,2133.69
7,39363,1,2106.55,1.0,2106.55
8,77818,1,2094.44,1.0,2094.44
9,55495,1,2066.34,1.0,2066.34


In [58]:
# identificar regiões com maior ticket médio
pandas_sql("""
SELECT
    customer_cep,
    COUNT(DISTINCT customer_id) AS num_customers,
    AVG(avg_ticket) AS region_avg_ticket,
    AVG(total_orders) AS region_avg_orders,
    SUM(total_spent) AS region_total_spent
FROM delta_scan('../delta_lake/gold/gold_customers_segmented')
GROUP BY customer_cep
ORDER BY region_avg_ticket DESC
LIMIT 10;
""")

# Essa consulta te mostrará as 10 regiões (CEP prefixado) com:
# Maior ticket médio por cliente (region_avg_ticket)
# Número médio de pedidos (region_avg_orders)
# Quantidade de clientes
# Total de compras realizadas na região

,customer_cep,num_customers,region_avg_ticket,region_avg_orders,region_total_spent
0,38447,1,2416.00,1.0,2416.00
1,02635,1,2328.40,1.0,2328.40
2,58765,1,2324.99,1.0,2324.99
3,57275,1,2269.98,1.0,2269.98
4,74455,1,2267.28,1.0,2267.28
5,68618,1,2252.66,1.0,2252.66
6,79906,1,2133.69,1.0,2133.69
7,39363,1,2106.55,1.0,2106.55
8,77818,1,2094.44,1.0,2094.44
9,55495,1,2066.34,1.0,2066.34


## Geolocalização Rule

✅ Business Rule: Expand sellers to regions with high demand to balance product supply.

✅ Regra de Negócio: Expandir vendedores para regiões com alta demanda para equilibrar a oferta de produtos.

In [14]:
pandas_sql("""
SELECT
    customer_cep,
    customer_city,
    customer_state,
    avg_latitude,
    avg_longitude,
    COUNT(DISTINCT customer_unique_id) AS total_customers
FROM delta_scan('../delta_lake/silver/aggregated_customers')
GROUP BY
    customer_cep,
    customer_city,
    customer_state,
    avg_latitude,
    avg_longitude
LIMIT 10;

""")

,customer_cep,customer_city,customer_state,avg_latitude,avg_longitude,total_customers
0,71015,brasilia,DF,-15.823198,-47.987108,18
1,71670,brasilia,DF,-15.819915,-47.814722,6
2,88508,lages,SC,-27.805198,-50.313912,10
3,90240,porto alegre,RS,-30.004097,-51.196319,18
4,42800,camacari,BA,-12.701936,-38.324333,10
5,60410,fortaleza,CE,-3.763646,-38.539839,8
6,64049,teresina,PI,-5.068139,-42.788688,21
7,06140,osasco,SP,-23.576526,-46.800874,11
8,09185,santo andre,SP,-23.682941,-46.535487,22
9,09941,diadema,SP,-23.678007,-46.602978,19


In [25]:
# Apoiar decisões estratégicas de expansão de vendedores com base em regiões com alta demanda e oferta limitada.
pandas_sql("""
SELECT
    ac.customer_cep,
    ac.customer_city,
    ac.customer_state,
    ac.avg_latitude,
    ac.avg_longitude,
    COUNT(DISTINCT ac.customer_unique_id) AS total_customers,
    COUNT(DISTINCT o.order_id) AS total_orders,
    COUNT(DISTINCT oi.seller_id) AS distinct_sellers,
    SUM(p.payment_value) AS total_sales_value,
    CASE 
        WHEN COUNT(DISTINCT oi.seller_id) = 0 THEN NULL
        ELSE ROUND(
            CAST(COUNT(DISTINCT o.order_id) AS FLOAT) / COUNT(DISTINCT oi.seller_id),
            2
        )
    END AS demand_supply_ratio
FROM delta_scan('../delta_lake/silver/aggregated_customers') ac
LEFT JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o ON ac.customer_id = o.customer_id
LEFT JOIN delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
LEFT JOIN delta_scan('../delta_lake/silver/payments_silver') p ON o.order_id = p.order_id
GROUP BY
    ac.customer_cep,
    ac.customer_city,
    ac.customer_state,
    ac.avg_latitude,
    ac.avg_longitude
ORDER BY total_orders DESC
LIMIT 5;
""")

# total_customers:	Total de clientes únicos que compraram nessa região
# total_orders:	Total de pedidos realizados por esses clientes
# distinct_sellers:	Número de vendedores distintos que atenderam os pedidos
# total_sales_value:	Valor total de vendas (somatório dos pagamentos) naquela região
# demand_supply_ratio:	Indicador de pressão de demanda: quanto maior, maior a demanda por vendedor

# coluna demand_supply_ratio:
# Se o valor for 1.00 → Cada vendedor atendeu 1 pedido (baixa demanda por vendedor).
# Se o valor for 3.50 → Cada vendedor atendeu, em média, 3,5 pedidos (alta demanda por vendedor).

,customer_cep,customer_city,customer_state,avg_latitude,avg_longitude,total_customers,total_orders,distinct_sellers,total_sales_value,demand_supply_ratio
0,22790,rio de janeiro,RJ,-23.012987,-43.465520,136,142,118,28290.74,1.20
1,24220,niteroi,RJ,-22.903245,-43.107024,114,124,106,25334.92,1.17
2,22793,rio de janeiro,RJ,-23.000619,-43.404718,119,121,100,27613.07,1.21
3,24230,niteroi,RJ,-22.878637,-43.086096,113,117,98,25205.68,1.19
4,22775,rio de janeiro,RJ,-22.969148,-43.383065,107,110,88,35665.34,1.25


In [29]:
gold_pipeline_sql(query="""
SELECT
    ac.customer_cep,
    ac.customer_city,
    ac.customer_state,
    ac.avg_latitude,
    ac.avg_longitude,
    COUNT(DISTINCT ac.customer_unique_id) AS total_customers,
    COUNT(DISTINCT o.order_id) AS total_orders,
    COUNT(DISTINCT oi.seller_id) AS distinct_sellers,
    SUM(p.payment_value) AS total_sales_value,
    CASE 
        WHEN COUNT(DISTINCT oi.seller_id) = 0 THEN NULL
        ELSE ROUND(
            CAST(COUNT(DISTINCT o.order_id) AS FLOAT) / COUNT(DISTINCT oi.seller_id),
            2
        )
    END AS demand_supply_ratio
FROM delta_scan('../delta_lake/silver/aggregated_customers') ac
LEFT JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o ON ac.customer_id = o.customer_id
LEFT JOIN delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
LEFT JOIN delta_scan('../delta_lake/silver/payments_silver') p ON o.order_id = p.order_id
GROUP BY
    ac.customer_cep,
    ac.customer_city,
    ac.customer_state,
    ac.avg_latitude,
    ac.avg_longitude
ORDER BY total_orders DESC;
""", gold_table_name="regional_demand_supply_balance")

2025-04-07 11:57:25,893 - INFO - Executando transformação na tabela 'regional_demand_supply_balance' para a camada GOLD
2025-04-07 11:57:26,093 - INFO - [OK] Tabela 'regional_demand_supply_balance' processada com sucesso.


In [30]:
pandas_sql("SELECT * FROM delta_scan('../delta_lake/gold/regional_demand_supply_balance') LIMIT 5")

,customer_cep,customer_city,customer_state,avg_latitude,avg_longitude,total_customers,total_orders,distinct_sellers,total_sales_value,demand_supply_ratio
0,22790,rio de janeiro,RJ,-23.012987,-43.465520,136,142,118,28290.74,1.20
1,24220,niteroi,RJ,-22.903245,-43.107024,114,124,106,25334.92,1.17
2,22793,rio de janeiro,RJ,-23.000619,-43.404718,119,121,100,27613.07,1.21
3,24230,niteroi,RJ,-22.878637,-43.086096,113,117,98,25205.68,1.19
4,22775,rio de janeiro,RJ,-22.969148,-43.383065,107,110,88,35665.34,1.25


## Order Items Rule

✅ Business Rule: Products with low profit margins and high shipping costs should be re-evaluated in the catalog.

✅ Regra de Negócio: Produtos com baixa margem de lucro e alto custo de frete devem ser reavaliados no catálogo.

In [32]:
pandas_sql("SELECT * FROM delta_scan('../delta_lake/silver/order_items_silver') LIMIT 5")

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [33]:
pandas_sql("SELECT * FROM delta_scan('../delta_lake/silver/products_silver') LIMIT 5")

,product_id,product_category,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


In [45]:
pandas_sql("""
SELECT 
    p.product_id,
    p.product_category,
    ROUND(AVG(oi.price), 2) AS avg_price, -- preço médio
    ROUND(AVG(oi.freight_value), 2) AS avg_freight, -- média do frete do produto
    COUNT(oi.order_id) AS total_sales, -- total de vendas
    AVG(p.product_weight_g) AS avg_weight, -- média de peso do produto
    ROUND((AVG(oi.freight_value) / AVG(oi.price)) * 100, 2) AS freight_pct_price, -- percentual do frete em relação ao preço médio do produto
    CASE 
        WHEN (AVG(oi.freight_value) / AVG(oi.price)) * 100 >= 30 THEN True
        ELSE False
    END AS critical_product
FROM delta_scan('../delta_lake/silver/order_items_silver') oi
JOIN delta_scan('../delta_lake/silver/products_silver') p 
    ON oi.product_id = p.product_id
GROUP BY 
    p.product_id, 
    p.product_category
LIMIT 10;
""")

,product_id,product_category,avg_price,avg_freight,total_sales,avg_weight,freight_pct_price,critical_product
0,5ed9eaf534f6936b51d0b6c5e4d5c2e9,cool_stuff,24.98,18.74,21,600.0,75.01,True
1,777d2e438a1b645f3aec9bd57e92672c,cama_mesa_banho,69.87,16.43,94,950.0,23.51,False
2,13fcfc313dfb2217e5ee3000a702f9ef,esporte_lazer,74.90,15.95,2,450.0,21.29,False
3,3fbc0ef745950c7932d5f2a446189725,beleza_saude,65.75,26.07,150,350.0,39.65,True
4,5e21d5cab5d33e770d8150a4ee6117db,relogios_presentes,49.00,13.36,12,200.0,27.27,False
5,85d619318aa201aca57171aabeed0048,esporte_lazer,14.00,10.40,4,150.0,74.29,True
6,779559842fc122d55edbd03153f35e81,esporte_lazer,27.40,12.84,8,200.0,46.84,True
7,57f2bc497c1a3ebe41ba7a06d78ed159,relogios_presentes,54.97,13.24,38,250.0,24.09,False
8,a8115d34a4b7b74388da609692162c37,informatica_acessorios,32.90,18.69,24,250.0,56.82,True
9,77fdeb8d4f7502f75b6779205f30a875,cama_mesa_banho,118.83,57.85,9,12250.0,48.68,True


In [46]:
gold_pipeline_sql(query="""
SELECT 
    p.product_id,
    p.product_category,
    ROUND(AVG(oi.price), 2) AS avg_price, -- preço médio
    ROUND(AVG(oi.freight_value), 2) AS avg_freight, -- média do frete do produto
    COUNT(oi.order_id) AS total_sales, -- total de vendas
    AVG(p.product_weight_g) AS avg_weight, -- média de peso do produto
    ROUND((AVG(oi.freight_value) / AVG(oi.price)) * 100, 2) AS freight_pct_price, -- percentual do frete em relação ao preço médio do produto
    CASE 
        WHEN (AVG(oi.freight_value) / AVG(oi.price)) * 100 >= 30 THEN True
        ELSE False
    END AS critical_product
FROM delta_scan('../delta_lake/silver/order_items_silver') oi
JOIN delta_scan('../delta_lake/silver/products_silver') p 
    ON oi.product_id = p.product_id
GROUP BY 
    p.product_id, 
    p.product_category;
""", gold_table_name="gold_product_profitability_analysis")

2025-04-07 22:05:47,183 - INFO - Executando transformação na tabela 'gold_product_profitability_analysis' para a camada GOLD
2025-04-07 22:05:47,314 - INFO - [OK] Tabela 'gold_product_profitability_analysis' processada com sucesso.


## Pagamentos Rule

✅ Business Rule: Encourage payment methods that generate a higher average ticket and lower cancellation rate.

✅ Regra de Negócio: Estimular métodos de pagamento que geram maior ticket médio e menor taxa de cancelamento.


In [6]:
# gold_payment_performance
pandas_sql("""
SELECT 
  op.payment_type,
  AVG(op.payment_value) AS avg_payment_value,
  COUNT(o.order_id) AS total_orders,
  SUM(CASE WHEN o.order_status = 'cancelled' THEN 1 ELSE 0 END) AS cancelled_orders,
  SUM(CASE WHEN o.order_status = 'cancelled' THEN 1 ELSE 0 END) * 1.0 / COUNT(o.order_id) AS cancellation_rate
FROM 
  delta_scan('../delta_lake/silver/payments_silver') op
  JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o ON op.order_id = o.order_id
GROUP BY 
  op.payment_type
LIMIT 10;
""")

,payment_type,avg_payment_value,total_orders,cancelled_orders,cancellation_rate
0,not_defined,0.000000,3,0.0,0.0
1,credit_card,163.319021,76795,0.0,0.0
2,boleto,145.034435,19784,0.0,0.0
3,voucher,65.703354,5775,0.0,0.0
4,debit_card,142.570170,1529,0.0,0.0


In [4]:
# v3
pandas_sql("""
SELECT 
  payment_type,
  ROUND(AVG(avg_payment_value), 2) AS avg_payment_value,
  SUM(total_orders) AS total_orders,
  SUM(cancelled_orders) AS cancelled_orders,
  SUM(cancelled_orders) * 1.0 / SUM(total_orders) AS cancellation_rate,
  ROUND(AVG(avg_order_value), 2) AS avg_order_value
FROM (
  SELECT 
    op.payment_type,
    ROUND(AVG(op.payment_value), 2) AS avg_payment_value,
    COUNT(o.order_id) AS total_orders,
    SUM(CASE WHEN o.order_status = 'cancelled' THEN 1 ELSE 0 END) AS cancelled_orders,
    SUM(CASE WHEN o.order_status = 'cancelled' THEN 1 ELSE 0 END) * 1.0 / COUNT(o.order_id) AS cancellation_rate,
    ROUND(AVG(oi.price + oi.freight_value), 2) AS avg_order_value
  FROM 
    delta_scan('../delta_lake/silver/payments_silver') op
    JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o ON op.order_id = o.order_id
    JOIN delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
    JOIN delta_scan('../delta_lake/silver/sellers_silver') s ON oi.seller_id = s.seller_id
  GROUP BY 
    op.payment_type, s.seller_id, s.seller_city, s.seller_state
) AS subquery
GROUP BY 
  payment_type
-- LIMIT 5;
""")

,payment_type,avg_payment_value,total_orders,cancelled_orders,cancellation_rate,avg_order_value
0,boleto,208.64,22867.0,0.0,0.0,170.88
1,debit_card,168.49,1691.0,0.0,0.0,154.24
2,voucher,85.07,6274.0,0.0,0.0,143.79
3,credit_card,228.01,86769.0,0.0,0.0,198.65


In [7]:
# v4
pandas_sql("""
SELECT 
  op.payment_type,
  ROUND(AVG(op.payment_value), 2) AS avg_payment_value,
  COUNT(DISTINCT o.order_id) AS total_orders,
  SUM(CASE WHEN o.order_status = 'cancelled' THEN 1 ELSE 0 END) AS cancelled_orders,
  SUM(CASE WHEN o.order_status = 'cancelled' THEN 1 ELSE 0 END) * 1.0 / COUNT(DISTINCT o.order_id) AS cancellation_rate,
  ROUND(AVG(oi.price + oi.freight_value), 2) AS avg_order_value
FROM 
  delta_scan('../delta_lake/silver/payments_silver') op
  JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o ON op.order_id = o.order_id
  JOIN delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
GROUP BY 
  op.payment_type
ORDER BY 
  avg_order_value DESC, cancellation_rate ASC;
""")

,payment_type,avg_payment_value,total_orders,cancelled_orders,cancellation_rate,avg_order_value
0,credit_card,179.66,75991,0.0,0.0,146.64
1,debit_card,149.93,1521,0.0,0.0,127.33
2,voucher,64.69,3766,0.0,0.0,125.13
3,boleto,177.54,19614,0.0,0.0,124.29


In [8]:
gold_pipeline_sql("""
SELECT 
  op.payment_type,
  ROUND(AVG(op.payment_value), 2) AS avg_payment_value,
  COUNT(DISTINCT o.order_id) AS total_orders,
  SUM(CASE WHEN o.order_status = 'cancelled' THEN 1 ELSE 0 END) AS cancelled_orders,
  SUM(CASE WHEN o.order_status = 'cancelled' THEN 1 ELSE 0 END) * 1.0 / COUNT(DISTINCT o.order_id) AS cancellation_rate,
  ROUND(AVG(oi.price + oi.freight_value), 2) AS avg_order_value
FROM 
  delta_scan('../delta_lake/silver/payments_silver') op
  JOIN delta_scan('../delta_lake/silver/orders_full_data_silver') o ON op.order_id = o.order_id
  JOIN delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
GROUP BY 
  op.payment_type
ORDER BY 
  avg_order_value DESC, cancellation_rate ASC;
""", gold_table_name="gold_payment_performance")

2025-04-15 21:26:32,251 - INFO - Executando transformação na tabela 'gold_payment_performance' para a camada GOLD
2025-04-15 21:26:34,184 - INFO - [OK] Tabela 'gold_payment_performance' processada com sucesso.


## Avaliações Rule (order_reviews)

✅ Business Rule: Sellers with ratings below 3 stars must be notified for corrective action.

✅ Regra de Negócio: Vendedores com avaliações abaixo de 3 estrelas devem ser notificados para ações corretivas.


In [12]:
# consulta para identificar os vendedores com avalições abaixo de 3 estrelas
pandas_sql("""
SELECT 
    s.seller_id,
    s.seller_city,
    s.seller_state,
    ROUND(AVG(orv.review_score), 2) AS avg_score_review
FROM 
    delta_scan('../delta_lake/silver/sellers_silver') s
JOIN 
    delta_scan('../delta_lake/silver/order_items_silver') oi ON s.seller_id = oi.seller_id
JOIN 
    delta_scan('../delta_lake/silver/order_reviews') orv ON oi.order_id = orv.order_id
GROUP BY 
    s.seller_id, s.seller_city, s.seller_state
HAVING 
    AVG(orv.review_score) < 3
LIMIT 10;
""")

,seller_id,seller_city,seller_state,avg_score_review
0,2dd1fb6f0e42a2243b8a01e40cd41216,curitiba,PR,2.60
1,972d0f9cf61b499a4812cf0bfa3ad3c4,brusque,SC,2.96
2,ec21206b7894c503788a684a7bc822a6,armacao dos buzios,RJ,2.67
3,7fdb0720c8d7c9075538b365dc8c3a22,toledo,PR,2.40
4,9c3a1c416c765687bc79a04113269929,carmo da mata,MG,1.00
5,f7df46c1e0ec44eed5c6726478da4a17,novo hamburgo,RS,1.00
6,ea1776927851da2532aa70d6a32c095a,sao paulo,SP,2.67
7,6f1a1263039c76e68f40a8e536b1da6a,sao paulop,SP,2.14
8,5d043cd5512d4bd2f88e5ccdd5736c38,sao paulo,SP,2.00
9,ebef8d8c92b4fd3226e5d407bbff21ac,ribeirao preto,SP,2.67


In [13]:
gold_pipeline_sql("""
SELECT 
    s.seller_id,
    s.seller_city,
    s.seller_state,
    ROUND(AVG(orv.review_score), 2) AS avg_score_review
FROM 
    delta_scan('../delta_lake/silver/sellers_silver') s
JOIN 
    delta_scan('../delta_lake/silver/order_items_silver') oi ON s.seller_id = oi.seller_id
JOIN 
    delta_scan('../delta_lake/silver/order_reviews') orv ON oi.order_id = orv.order_id
GROUP BY 
    s.seller_id, s.seller_city, s.seller_state
HAVING 
    AVG(orv.review_score) < 3;
""", gold_table_name="gold_sellers_order_reviews_score")

2025-04-15 21:39:11,525 - INFO - Executando transformação na tabela 'gold_sellers_order_reviews_score' para a camada GOLD
2025-04-15 21:39:11,703 - INFO - [OK] Tabela 'gold_sellers_order_reviews_score' processada com sucesso.


## Pedidos Rule (orders)

✅ Business Rule: Monitor orders with recurring delays and penalize carriers that fail to meet deadlines.

✅ Regra de Negócio: Monitorar pedidos com atraso recorrente e penalizar transportadoras que descumprirem prazos.

In [24]:
pandas_sql("""
SELECT 
    oi.seller_id,
    s.seller_city,
    s.seller_state,
    COUNT(o.order_id) AS total_pedidos,
    SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) AS pedidos_atrasados
FROM 
    delta_scan('../delta_lake/silver/orders_full_data_silver') o
JOIN 
    delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
JOIN 
    delta_scan('../delta_lake/silver/sellers_silver') s ON oi.seller_id = s.seller_id
GROUP BY 
    oi.seller_id, s.seller_city, s.seller_state
HAVING 
    SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) > 0
LIMIT 5;
""")

,seller_id,seller_city,seller_state,total_pedidos,pedidos_atrasados
0,8cbac7e12637ed9cffa18c7875207478,jaragua do sul,SC,82,4.0
1,5656537e588803a555b8eb41f07a944b,brasilia,DF,188,14.0
2,c3cfdc648177fdbbbb35635a37472c53,curitiba,PR,307,7.0
3,058fd0aa2bfdb2274e05e1ae971dabb6,santo andre,SP,64,5.0
4,dbc22125167c298ef99da25668e1011f,borda da mata,MG,429,10.0


In [28]:
# v2
pandas_sql("""
SELECT 
    oi.seller_id,
    s.seller_city,
    s.seller_state,
    COUNT(o.order_id) AS total_orders,
    SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders,
    ROUND(SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) * 100.0 / COUNT(o.order_id), 2) AS delayed_rate_percent,
    ROUND(AVG(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN DATE_DIFF('day', o.order_estimated_delivery_date, o.order_delivered_customer_date) ELSE 0 END), 2) AS avg_delay_days,
    MAX(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN o.order_delivered_customer_date ELSE NULL END) AS last_order_delayed
FROM 
    delta_scan('../delta_lake/silver/orders_full_data_silver') o
JOIN 
    delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
JOIN 
    delta_scan('../delta_lake/silver/sellers_silver') s ON oi.seller_id = s.seller_id
GROUP BY 
    oi.seller_id, s.seller_city, s.seller_state
HAVING 
    SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) > 0
LIMIT 5;
""")

,seller_id,seller_city,seller_state,total_orders,delayed_orders,delayed_rate_percent,avg_delay_days,last_order_delayed
0,9d7a1d34a5052409006425275ba1c2b4,franca,SP,16,1.0,6.25,0.19,2018-03-26 15:51:47
1,df560393f3a51e74553ab94004ba5c87,loanda,PR,29,2.0,6.90,1.41,2018-04-02 22:32:10
2,7040e82f899a04d1b434b795a43b4617,sao paulo,SP,228,20.0,8.77,0.55,2018-08-13 15:41:47
3,6560211a19b47992c3666cc44a7e94c0,sao paulo,SP,2033,124.0,6.10,0.38,2018-09-14 17:11:19
4,2a84855fd20af891be03bc5924d2b453,belo horizonte,MG,168,5.0,2.98,0.60,2018-09-12 15:51:39


In [29]:
gold_pipeline_sql("""
SELECT 
    oi.seller_id,
    s.seller_city,
    s.seller_state,
    COUNT(o.order_id) AS total_orders,
    SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders,
    ROUND(SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) * 100.0 / COUNT(o.order_id), 2) AS delayed_rate_percent,
    ROUND(AVG(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN DATE_DIFF('day', o.order_estimated_delivery_date, o.order_delivered_customer_date) ELSE 0 END), 2) AS avg_delay_days,
    MAX(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN o.order_delivered_customer_date ELSE NULL END) AS last_order_delayed
FROM 
    delta_scan('../delta_lake/silver/orders_full_data_silver') o
JOIN 
    delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
JOIN 
    delta_scan('../delta_lake/silver/sellers_silver') s ON oi.seller_id = s.seller_id
GROUP BY 
    oi.seller_id, s.seller_city, s.seller_state
HAVING 
    SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) > 0
""", gold_table_name="gold_sellers_shipment_metrics")

2025-04-15 22:17:34,632 - INFO - Executando transformação na tabela 'gold_sellers_shipment_metrics' para a camada GOLD
2025-04-15 22:17:34,819 - INFO - [OK] Tabela 'gold_sellers_shipment_metrics' processada com sucesso.


## Produtos Rule (products)

✅ Business Rule: Products with the highest sales volume should have priority in stock to avoid stockouts.

✅ Regra de Negócio: Produtos com maior volume de vendas devem ter prioridade no estoque para evitar rupturas.

In [12]:
pandas_sql("""
SELECT * FROM delta_scan('../delta_lake/silver/products_silver') LIMIT 5
""")

,product_id,product_category,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


In [15]:
# gold_product_priority - v1
pandas_sql("""
SELECT 
  p.product_id,
  p.product_category,
  SUM(oi.price) AS total_sales,
  COUNT(oi.order_id) AS sales_count,
  ROUND(SUM(oi.price) / COUNT(oi.order_id), 2) AS avg_price_per_sale, 
  RANK() OVER (ORDER BY SUM(oi.price) DESC) AS sales_priority -- ranking de prioridade em ordem
FROM 
  delta_scan('../delta_lake/silver/products_silver') p
  JOIN delta_scan('../delta_lake/silver/order_items_silver') oi ON p.product_id = oi.product_id
GROUP BY 
  p.product_id, p.product_category
ORDER BY 
  total_sales DESC
LIMIT 5;
""")

,product_id,product_category,total_sales,sales_count,avg_price_per_sale,sales_priority
0,bb50f2e236e5eea0100680137654686c,beleza_saude,63885.00,195,327.62,1
1,6cdd53843498f92890544667809f1595,beleza_saude,54730.20,156,350.83,2
2,d6160fb7873f184099d9bc95e30376af,pcs,48899.34,35,1397.12,3
3,d1c427060a0f73f6b889a5c7c61f2ac4,informatica_acessorios,47214.51,343,137.65,4
4,99a4788cb24856965c36a24e339b6058,cama_mesa_banho,43025.56,488,88.17,5


In [16]:
gold_pipeline_sql("""
SELECT 
  p.product_id,
  p.product_category,
  SUM(oi.price) AS total_sales,
  COUNT(oi.order_id) AS sales_count,
  ROUND(SUM(oi.price) / COUNT(oi.order_id), 2) AS avg_price_per_sale, 
  RANK() OVER (ORDER BY SUM(oi.price) DESC) AS sales_priority -- ranking de prioridade em ordem
FROM 
  delta_scan('../delta_lake/silver/products_silver') p
  JOIN delta_scan('../delta_lake/silver/order_items_silver') oi ON p.product_id = oi.product_id
GROUP BY 
  p.product_id, p.product_category
ORDER BY 
  total_sales DESC;
""", gold_table_name="gold_product_inventory_priority")

2025-04-16 11:57:06,944 - INFO - Executando transformação na tabela 'gold_product_inventory_priority' para a camada GOLD
2025-04-16 11:57:08,674 - INFO - [OK] Tabela 'gold_product_inventory_priority' processada com sucesso.


## Leads Qualificados Rule (leads_qualified)

✅ Business Rule: Qualified leads originating from high-performance campaigns must receive priority in service.

✅ Regra de Negócio: Leads qualificados com origem em campanhas de alto desempenho devem receber prioridade no atendimento.

In [17]:
pandas_sql("""
SELECT * FROM delta_scan('../delta_lake/silver/aggregated_leads_silver') LIMIT 5;
""")

,mql_id,first_contact_date,came_from,won_date,business_segment,lead_type,days_to_convert
0,5420aad7fec3549a85876ba1c529bd84,2018-02-21,organic_search,2018-02-26 19:58:54,pet,online_medium,5
1,a555fb36b9368110ede0f043dfc3b9a0,2018-04-04,referral,2018-05-08 20:17:59,car_accessories,industry,34
2,327174d3648a2d047e8940d7d15204ca,2018-04-03,organic_search,2018-06-05 17:27:23,home_appliances,online_big,63
3,f5fee8f7da74f4887f5bcae2bafb6dd6,2018-01-14,paid_search,2018-01-17 13:51:03,food_drink,online_small,3
4,ffe640179b554e295c167a2f6be528e0,2017-10-09,unknown,2018-07-03 20:17:45,home_appliances,industry,267


In [18]:
pandas_sql("""
SELECT * FROM delta_scan('../delta_lake/silver/leads_qualified_silver') LIMIT 5;
""")

,mql_id,landing_page_id,first_contact_date,came_from
0,dac32acd4db4c29c230538b72f8dd87d,88740e65d5d6b056e0cda098e1ea6313,2018-02-01,social
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,007f9098284a86ee80ddeb25d53e0af8,2017-10-20,paid_search
2,b4bc852d233dfefc5131f593b538befa,a7982125ff7aa3b2054c6e44f9d28522,2018-03-22,organic_search
3,6be030b81c75970747525b843c1ef4f8,d45d558f0daeecf3cccdffe3c59684aa,2018-01-22,email
4,5420aad7fec3549a85876ba1c529bd84,b48ec5f3b04e9068441002a19df93c6c,2018-02-21,organic_search


In [27]:
pandas_sql("""
WITH leads_origin AS (
  SELECT 
    came_from,
    COUNT(mql_id) AS total_leads
  FROM 
    delta_scan('../delta_lake/silver/leads_qualified_silver')
  GROUP BY 
    came_from
),
ranked_leads AS (
  SELECT 
    mql_id,
    came_from,
    ROW_NUMBER() OVER (ORDER BY came_from) AS row_num
  FROM 
    delta_scan('../delta_lake/silver/leads_qualified_silver')
)
SELECT 
  l.mql_id,
  l.first_contact_date,
  l.landing_page_id,
  l.came_from,
  lo.total_leads,
  -- calcular prioridade baseada na frequência
  CASE 
    WHEN lo.total_leads > (SELECT AVG(total_leads) FROM leads_origin) THEN 'high'
    WHEN lo.total_leads = (SELECT AVG(total_leads) FROM leads_origin) THEN 'median'
    ELSE 'low'
  END AS priority
FROM 
  delta_scan('../delta_lake/silver/leads_qualified_silver') l
JOIN 
  leads_origin lo ON l.came_from = lo.came_from
ORDER BY 
  lo.total_leads DESC
LIMIT 5;
""")

,mql_id,first_contact_date,landing_page_id,came_from,total_leads,priority
0,28bdfd5f057764b54c38770f95c69f2f,2018-01-14,22c29808c4f815213303f8933030604c,organic_search,2296,high
1,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2296,high
2,988c9372560f30a5f28f92640bb2b3da,2018-05-23,73f31a40697cc90a86c1273563ac230e,organic_search,2296,high
3,2f838cade4a6012a6cb1016d1d8d95ed,2017-11-10,aeac92c0f5ae22a04ed3b746cce3a1b6,organic_search,2296,high
4,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,2296,high


In [28]:
gold_pipeline_sql("""
WITH leads_origin AS (
  SELECT 
    came_from,
    COUNT(mql_id) AS total_leads
  FROM 
    delta_scan('../delta_lake/silver/leads_qualified_silver')
  GROUP BY 
    came_from
),
ranked_leads AS (
  SELECT 
    mql_id,
    came_from,
    ROW_NUMBER() OVER (ORDER BY came_from) AS row_num
  FROM 
    delta_scan('../delta_lake/silver/leads_qualified_silver')
)
SELECT 
  l.mql_id,
  l.first_contact_date,
  l.landing_page_id,
  l.came_from,
  lo.total_leads,
  -- calcular prioridade baseada na frequência
  CASE 
    WHEN lo.total_leads > (SELECT AVG(total_leads) FROM leads_origin) THEN 'high'
    WHEN lo.total_leads = (SELECT AVG(total_leads) FROM leads_origin) THEN 'median'
    ELSE 'low'
  END AS priority
FROM 
  delta_scan('../delta_lake/silver/leads_qualified_silver') l
JOIN 
  leads_origin lo ON l.came_from = lo.came_from
ORDER BY 
  lo.total_leads DESC;
""", gold_table_name="gold_qualified_leads_priority")

2025-04-16 12:20:26,227 - INFO - Executando transformação na tabela 'gold_qualified_leads_priority' para a camada GOLD
2025-04-16 12:20:26,338 - INFO - [OK] Tabela 'gold_qualified_leads_priority' processada com sucesso.


## sales_by_category 

Receita total e quantidade vendida por categoria.

In [47]:
pandas_sql("""
SELECT 
    p.product_category,
    ROUND(SUM(oi.price), 2) AS total_revenue,
    COUNT(oi.order_id) AS total_quantity_sold,
    ROUND(AVG(oi.price), 2) AS average_price_per_item
FROM 
    delta_scan('../delta_lake/silver/order_items_silver') oi
JOIN 
    delta_scan('../delta_lake/silver/products_silver') p ON oi.product_id = p.product_id
GROUP BY 
    p.product_category
LIMIT 10;
""")

,product_category,total_revenue,total_quantity_sold,average_price_per_item
0,cama_mesa_banho,1036988.68,11115,93.30
1,fashion_roupa_feminina,2803.64,48,58.41
2,fashion_esporte,2119.51,30,70.65
3,agro_industria_e_comercio,72530.47,212,342.12
4,sinalizacao_e_seguranca,21509.23,199,108.09
5,construcao_ferramentas_construcao,144677.59,929,155.73
6,brinquedos,483946.60,4117,117.55
7,casa_conforto_2,760.27,30,25.34
8,utilidades_domesticas,632248.66,6964,90.79
9,telefonia,323667.53,4545,71.21


In [48]:
# sales_by_category
gold_pipeline_sql("""
SELECT 
    p.product_category,
    ROUND(SUM(oi.price), 2) AS total_revenue,
    COUNT(oi.order_id) AS total_quantity_sold,
    ROUND(AVG(oi.price), 2) AS average_price_per_item
FROM 
    delta_scan('../delta_lake/silver/order_items_silver') oi
JOIN 
    delta_scan('../delta_lake/silver/products_silver') p ON oi.product_id = p.product_id
GROUP BY 
    p.product_category;
""", gold_table_name="sales_by_category")

2025-04-16 21:31:06,227 - INFO - Executando transformação na tabela 'sales_by_category' para a camada GOLD
2025-04-16 21:31:06,356 - INFO - [OK] Tabela 'sales_by_category' processada com sucesso.


## freight_analysis

Média de frete por categoria e peso do produto.

In [59]:
pandas_sql("""
SELECT 
    p.product_category_name,
    SUM(oi.price) AS total_revenue,
    COUNT(oi.order_id) AS total_quantity_sold,
    ROUND(AVG(oi.price), 2) AS average_price_per_item,
    ROUND(AVG(oi.freight_value), 2) AS average_freight,
    ROUND(AVG(pr.product_weight_g), 2) AS average_product_weight_per_category
FROM 
    delta_scan('../delta_lake/silver/order_items_silver') oi
JOIN 
    delta_scan('../delta_lake/silver/products_silver') pr ON oi.product_id = pr.product_id
JOIN 
    delta_scan('../delta_lake/bronze/product_category_name_translation_bronze') p ON pr.product_category = p.product_category_name
GROUP BY 
    p.product_category_name
LIMIT 5;
""")

,product_category_name,total_revenue,total_quantity_sold,average_price_per_item,average_freight,average_product_weight_per_category
0,cool_stuff,635290.85,3796,167.36,22.14,2548.97
1,perfumaria,399124.87,3419,116.74,15.86,480.13
2,malas_acessorios,140429.98,1092,128.60,27.88,5774.55
3,bebes,411764.89,3065,134.34,22.30,3272.34
4,livros_interesse_geral,46856.88,553,84.73,16.63,760.52


In [60]:
gold_pipeline_sql("""
SELECT 
    p.product_category_name,
    SUM(oi.price) AS total_revenue,
    COUNT(oi.order_id) AS total_quantity_sold,
    ROUND(AVG(oi.price), 2) AS average_price_per_item,
    ROUND(AVG(oi.freight_value), 2) AS average_freight,
    ROUND(AVG(pr.product_weight_g), 2) AS average_product_weight_per_category
FROM 
    delta_scan('../delta_lake/silver/order_items_silver') oi
JOIN 
    delta_scan('../delta_lake/silver/products_silver') pr ON oi.product_id = pr.product_id
JOIN 
    delta_scan('../delta_lake/bronze/product_category_name_translation_bronze') p ON pr.product_category = p.product_category_name
GROUP BY 
    p.product_category_name;
""", gold_table_name="freight_analysis")

2025-04-16 21:43:18,776 - INFO - Executando transformação na tabela 'freight_analysis' para a camada GOLD
2025-04-16 21:43:18,913 - INFO - [OK] Tabela 'freight_analysis' processada com sucesso.


## seller_performance

Receita total e número de vendas por vendedor.

In [61]:
pandas_sql("""
SELECT
    s.seller_id,
    SUM(oi.price) AS total_revenue,
    COUNT(oi.order_id) AS total_sales
FROM
    delta_scan('../delta_lake/silver/order_items_silver') oi
JOIN 
    delta_scan('../delta_lake/silver/sellers_silver') s ON oi.seller_id = s.seller_id
GROUP BY
    s.seller_id
LIMIT 5;
""")

,seller_id,total_revenue,total_sales
0,48436dade18ac8b2bce089ec2a041202,12271.71,151
1,7c67e1448b00f6e969d365cea6b010ab,187923.89,1364
2,16090f2ca825584b5a147ab24aa30c86,25716.44,410
3,9f505651f4a6abe901a56cdc21508025,26361.82,430
4,f80edd2c5aaa505cc4b0a3b219abf4b8,9108.20,128


In [62]:
gold_pipeline_sql("""
SELECT
    s.seller_id,
    SUM(oi.price) AS total_revenue,
    COUNT(oi.order_id) AS total_sales
FROM
    delta_scan('../delta_lake/silver/order_items_silver') oi
JOIN 
    delta_scan('../delta_lake/silver/sellers_silver') s ON oi.seller_id = s.seller_id
GROUP BY
    s.seller_id;
""", gold_table_name="seller_performance")

2025-04-16 21:53:19,292 - INFO - Executando transformação na tabela 'seller_performance' para a camada GOLD
2025-04-16 21:53:19,378 - INFO - [OK] Tabela 'seller_performance' processada com sucesso.


## geo_sales

Receita e pedidos por cidade e estado.

In [66]:
pandas_sql("""
SELECT 
    c.customer_city,
    c.customer_state,
    SUM(oi.price) AS total_revenue,
    COUNT(o.order_id) AS total_orders
FROM 
    delta_scan('../delta_lake/silver/orders_full_data_silver') o
JOIN 
    delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
JOIN 
    delta_scan('../delta_lake/silver/customers_silver') c ON o.customer_id = c.customer_id
GROUP BY 
    c.customer_city, c.customer_state
LIMIT 10;
""")

,customer_city,customer_state,total_revenue,total_orders
0,timoteo,MG,7783.77,63
1,belo horizonte,MG,355611.13,3144
2,sumare,SP,21487.92,205
3,sao jose dos pinhais,PR,15124.05,143
4,fortaleza,CE,97868.06,728
5,bom principio,RS,1003.11,6
6,marialva,PR,2573.17,19
7,rio grande,RS,16516.78,132
8,piracaia,SP,2764.83,25
9,jambeiro,SP,329.26,6


In [67]:
gold_pipeline_sql("""
SELECT 
    c.customer_city,
    c.customer_state,
    SUM(oi.price) AS total_revenue,
    COUNT(o.order_id) AS total_orders
FROM 
    delta_scan('../delta_lake/silver/orders_full_data_silver') o
JOIN 
    delta_scan('../delta_lake/silver/order_items_silver') oi ON o.order_id = oi.order_id
JOIN 
    delta_scan('../delta_lake/silver/customers_silver') c ON o.customer_id = c.customer_id
GROUP BY 
    c.customer_city, c.customer_state;
""", gold_table_name="geo_sales")

2025-04-16 22:00:59,015 - INFO - Executando transformação na tabela 'geo_sales' para a camada GOLD
2025-04-16 22:00:59,163 - INFO - [OK] Tabela 'geo_sales' processada com sucesso.


## **Extra**: Camada Gold (Dados agregados para consumo)

[ x ] `sales_by_category`: Receita total e quantidade vendida por categoria.

[ x ]`freight_analysis`: Média de frete por categoria e peso do produto.

[ x ]`seller_performance`: Receita total e número de vendas por vendedor.

[ x ]`geo_sales`: Receita e pedidos por cidade e estado.